In [ ]:
import tensorflow as tf
assert tf.__version__.startswith('2')
  

import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

%matplotlib inline

In [ ]:
tf.__version__

'2.5.0'

In [ ]:


base_dir = "/content/drive/My Drive/Colab Notebooks/jpg"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
IMAGE_SIZE = 224
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

BATCH_SIZE1 = 5
BATCH_SIZE2=10
BATCH_SIZE3=20

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.1,samplewise_std_normalization=True)


train_generator1 = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE1, 
    subset='training')

val_generator1 = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE1, 
    subset='validation')

train_generator2 = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE2, 
    subset='training')

val_generator2 = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE2, 
    subset='validation')

train_generator3 = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE3, 
    subset='training')

val_generator3 = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE3, 
    subset='validation')



base_model1=tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

base_model2=tf.keras.applications.MobileNet(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

base_model3=tf.keras.applications.MobileNetV3(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')
optimizer1=tf.keras.optimizers.Adagrad()
optimizer2=tf.keras.optimizers.Adam(1e-5)
optimizer3=tf.keras.optimizers.RMSprop()




In [ ]:
for image_batch, label_batch in train_generator1:
  break
image_batch.shape, label_batch.shape


In [ ]:
print (train_generator1.class_indices)

labels = '\n'.join(sorted(train_generator1.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

In [ ]:
!cat labels.txt

In [ ]:

data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical", 
                                                 input_shape=(IMAGE_SIZE, 
                                                              IMAGE_SIZE,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
    
  ]
)

In [ ]:

def models(base_model,opt,train_generator,val_generator):
  base_model.trainable = False
  model = tf.keras.Sequential([
  data_augmentation,
  base_model,
    
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
    
  tf.keras.layers.GlobalAveragePooling2D(),
    
  tf.keras.layers.Dense(17, activation='softmax',kernel_regularizer="l2")
  ])

  model.compile(optimizer=tf.keras.optimizers.Adagrad(), 
                loss='categorical_crossentropy', 
                metrics=['accuracy'])
  model.summary()

  epochs = 20

  model.fit(train_generator, 
                      steps_per_epoch=len(train_generator), 
                      epochs=epochs, 
                      validation_data=val_generator, 
                      validation_steps=len(val_generator))

  

  base_model.trainable = True


  # Fine tune from this layer onwards
  fine_tune_at = 100

  # Freeze all the layers before the `fine_tune_at` layer
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

  model.compile(loss='categorical_crossentropy',
                optimizer = opt,
                metrics=['accuracy'])
  model.summary()

  history_fine = model.fit(train_generator, 
                          steps_per_epoch=len(train_generator), 
                          epochs=10, 
                          validation_data=val_generator, 
                          validation_steps=len(val_generator))


  acc = history_fine.history['accuracy']
  val_acc = history_fine.history['val_accuracy']

  loss = history_fine.history['loss']
  val_loss = history_fine.history['val_loss']

  plt.figure(figsize=(8, 8))
  plt.subplot(2, 1, 1)
  plt.plot(acc, label='Training Accuracy')
  plt.plot(val_acc, label='Validation Accuracy')
  plt.legend(loc='lower right')
  plt.ylabel('Accuracy')
  plt.ylim([min(plt.ylim()),1])
  plt.title('Training and Validation Accuracy')

  plt.subplot(2, 1, 2)
  plt.plot(loss, label='Training Loss')
  plt.plot(val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.ylabel('Cross Entropy')
  plt.ylim([0,4.0])
  plt.title('Training and Validation Loss')
  plt.xlabel('epoch')
  plt.show()

  return model

In [ ]:
hparam=[(base_model1,optimizer1,train_generator1,val_generator1),
        (base_model1,optimizer1,train_generator2,val_generator2),
        (base_model1,optimizer1,train_generator3,val_generator3),
        (base_model1,optimizer2,train_generator1,val_generator1),
        (base_model1,optimizer2,train_generator2,val_generator2),
        (base_model1,optimizer2,train_generator3,val_generator3),
        (base_model1,optimizer3,train_generator1,val_generator1),
        (base_model1,optimizer3,train_generator2,val_generator2),
        (base_model1,optimizer3,train_generator3,val_generator3),

        (base_model2,optimizer1,train_generator1,val_generator1),
        (base_model2,optimizer1,train_generator2,val_generator2),
        (base_model2,optimizer1,train_generator3,val_generator3),
        (base_model2,optimizer2,train_generator1,val_generator1),
        (base_model2,optimizer2,train_generator2,val_generator2),
        (base_model2,optimizer2,train_generator3,val_generator3),
        (base_model2,optimizer3,train_generator1,val_generator1),
        (base_model2,optimizer3,train_generator2,val_generator2),
        (base_model2,optimizer3,train_generator3,val_generator3),

        (base_model3,optimizer1,train_generator1,val_generator1),
        (base_model3,optimizer1,train_generator2,val_generator2),
        (base_model3,optimizer1,train_generator3,val_generator3),
        (base_model3,optimizer2,train_generator1,val_generator1),
        (base_model3,optimizer2,train_generator2,val_generator2),
        (base_model3,optimizer2,train_generator3,val_generator3),
        (base_model3,optimizer3,train_generator1,val_generator1),
        (base_model3,optimizer3,train_generator2,val_generator2),
        (base_model3,optimizer3,train_generator3,val_generator3)

        ]

Models=[]
for i in hparam:
  print("Model: {}".format(hparam.index(i)+1))
  Models.append(models(i[0],i[1],i[2],i[3]))


In [ ]:
Models[13].save("model.h5")
 
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)